In [2]:
!pip install pandas


In [3]:
import pandas as pd
import glob
import os

## Read file 

In [235]:
# Define the file path
file_path = '/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Trip_chain/trip_chain_dwell_time.csv'

# Read the CSV file into a DataFrame
trip_summary_df= pd.read_csv(file_path)

In [237]:
trip_summary_df = trip_summary_df.drop(columns=['age'])


In [243]:
# Define the file path
file_path = '/Users/aparnaj8/Box/Data (Matthew Rizzo)/DataAnalysis/FormattingQAChecks/REDCap_Formatting/REDCap_Format_Data/demographics.csv'

# Read the CSV file into a DataFrame
demo_summary_df= pd.read_csv(file_path)

In [245]:
demo_summary_df = demo_summary_df.rename(columns={'year': 'year_mapped'})


In [247]:
demo_summary_df.head()

,subj,subj_type,year_mapped,visit,dob,age,handedness,gender,education_years,education_degree,...,primarycar,driving_years,driving_miles,driving_days,driving_last,driving_environment,toldtostopdriving,toldtostopdriving_date,toldtostopdriving_who,toldtostopdriving_reason
0,RWRAD_001,mci,1,1,1942-02-13,79.0,right,male,mastersdegree,Physics,...,yes,63.0,100.0,sevendays,yesterday,urban,no,NaN,NaN,NaN
1,RWRAD_001,mci,2,4,NaN,80.0,NaN,male,mastersdegree,Physics,...,NaN,NaN,100.0,sevendays,yesterday,urban,no,NaN,NaN,NaN
2,RWRAD_001,mci,3,6,NaN,81.0,NaN,male,mastersdegree,Master's,...,NaN,NaN,150.0,sevendays,yesterday,urban,no,NaN,NaN,NaN
3,RWRAD_002,normalaging,1,1,1938-09-12,82.0,right,male,associatesdegree,90 hours college| numerous other certifications,...,yes,78.0,100.0,fivedays,yesterday,urban,no,NaN,NaN,NaN
4,RWRAD_002,normalaging,2,4,NaN,83.0,NaN,male,vocationalsomecollege,NaN,...,NaN,NaN,300.0,sevendays,yesterday,urban,no,NaN,NaN,NaN


In [249]:
# Define the file path
file_path =  '/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Speeding_Variable/high_speed_road_60mph_or_greater.csv'

# Read the CSV file into a DataFrame
high_speed_df= pd.read_csv(file_path)

In [251]:
high_speed_df.head()

,subj,drive,percentage_at_60mph_or_greater
0,RWRAD_001,40.0,54.816254
1,RWRAD_001,41.0,51.173539
2,RWRAD_001,54.0,73.404430
3,RWRAD_001,115.0,65.145791
4,RWRAD_001,116.0,64.764502


### Merge files

In [253]:
final_merged_df = trip_summary_df.merge(
    demo_summary_df[['subj', 'year_mapped', 'education_years', 'race', 'gender', 'age']],  # Select required columns from demo_summary_df
    on=['subj', 'year_mapped'],  # Merge on 'subj' and 'year_mapped' from both DataFrames
    how='left'  # Retain all rows from trip_summary_df
)


In [255]:
final_merged_df.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,week_number,start_location,end_location,time_diff_minutes,trip_chain,trip_chain_check,education_years,race,gender,age
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,11,NaN,NaN,NaN,NaN,NaN,mastersdegree,white,male,79.0
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,11,unc,social,4.350000,NaN,NaN,mastersdegree,white,male,79.0
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,11,unc,errand,12.650000,NaN,NaN,mastersdegree,white,male,79.0
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,11,unc,home,19.850000,NaN,NaN,mastersdegree,white,male,79.0
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,11,home,home,134.333333,1.0,1.0,mastersdegree,white,male,79.0


#### AM PM peak variable

In [259]:
final_merged_df['time_start_utc'] = pd.to_datetime(final_merged_df['time_start_utc'], errors='coerce')
final_merged_df['time_end_utc'] = pd.to_datetime(final_merged_df['time_end_utc'], errors='coerce')

In [261]:
final_merged_df['time_start_cst'] = pd.to_datetime(final_merged_df['time_start_cst'], errors='coerce')
final_merged_df['time_end_cst'] = pd.to_datetime(final_merged_df['time_end_cst'], errors='coerce')

In [263]:
import pandas as pd

def classify_time_of_day(timestamp):
    hour = timestamp.hour
    if 7 <= hour <= 9:
        return 'AM Peak'
    elif 16 <= hour <= 18:
        return 'PM Peak'
    else:
        return 'None'

# Example usage with a pandas dataframe
final_merged_df['AM_PM_peak'] = final_merged_df['time_start_cst'].apply(lambda x: classify_time_of_day(pd.to_datetime(x)))


In [265]:
final_merged_df.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,start_location,end_location,time_diff_minutes,trip_chain,trip_chain_check,education_years,race,gender,age,AM_PM_peak
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,NaN,NaN,NaN,NaN,NaN,mastersdegree,white,male,79.0,None
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,unc,social,4.350000,NaN,NaN,mastersdegree,white,male,79.0,None
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,unc,errand,12.650000,NaN,NaN,mastersdegree,white,male,79.0,None
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,unc,home,19.850000,NaN,NaN,mastersdegree,white,male,79.0,None
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,home,home,134.333333,1.0,1.0,mastersdegree,white,male,79.0,None


### Nighttime Daytime trip

In [267]:
import pandas as pd
from datetime import timedelta

# Define nighttime period (9 PM to 6 AM)
night_start = 21  # 9 PM
night_end = 6     # 6 AM

# Function to check if a specific time is nighttime
def is_nighttime(timestamp):
    hour = timestamp.hour
    return night_start <= hour or hour < night_end

# Function to calculate the percentage of the trip that is nighttime
def calculate_nighttime_percentage(row):
    # Convert start and end time to datetime
    start_time = pd.to_datetime(row['time_start_cst'])
    end_time = pd.to_datetime(row['time_end_cst'])

    # Calculate the total trip duration in minutes
    total_duration = (end_time - start_time).total_seconds() / 60  # in minutes

    # Initialize counters for nighttime minutes
    nighttime_minutes = 0

    # Iterate through each minute of the trip
    current_time = start_time
    while current_time <= end_time:
        if is_nighttime(current_time):
            nighttime_minutes += 1
        current_time += timedelta(minutes=1)

    # Calculate the percentage of time that is nighttime
    nighttime_percentage = (nighttime_minutes / total_duration) * 100
    return nighttime_percentage

# Apply the function to each row and assign "nighttime" or "daytime"
final_merged_df['night_trips'] = final_merged_df.apply(calculate_nighttime_percentage, axis=1)

# Create a new column 'day_or_night' based on the nighttime percentage
final_merged_df['day_or_night'] = final_merged_df['night_trips'].apply(
    lambda x: 'nighttime' if x >= 80 else 'daytime'
)



In [269]:
final_merged_df.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,time_diff_minutes,trip_chain,trip_chain_check,education_years,race,gender,age,AM_PM_peak,night_trips,day_or_night
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,NaN,NaN,NaN,mastersdegree,white,male,79.0,None,0.0,daytime
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,4.350000,NaN,NaN,mastersdegree,white,male,79.0,None,0.0,daytime
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,12.650000,NaN,NaN,mastersdegree,white,male,79.0,None,0.0,daytime
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,19.850000,NaN,NaN,mastersdegree,white,male,79.0,None,0.0,daytime
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,134.333333,1.0,1.0,mastersdegree,white,male,79.0,None,0.0,daytime


In [271]:
# Drop specified columns
columns_to_drop = ['home_distance', 'label', 'trip_chain', 'time_diff_minutes', 'night_trips']
final_merged_df = final_merged_df.drop(columns=columns_to_drop, errors='ignore')


In [278]:
final_merged_df.dtypes

subj                          object
drive                          int64
time_start_utc        datetime64[ns]
time_end_utc          datetime64[ns]
time_start_cst        datetime64[ns]
time_end_cst          datetime64[ns]
time_weekday                  object
duration_minutes             float64
distance_miles               float64
speed_mph_mean               float64
15_miles_from_home            object
25_miles_from_home            object
year                           int64
year_mapped                    int64
subj_type                     object
week_number                    int64
start_location                object
end_location                  object
trip_chain_check             float64
education_years               object
race                          object
gender                        object
age                          float64
AM_PM_peak                    object
day_or_night                  object
dtype: object

## Correct within 15 and 25 miles frm home variable

In [280]:
import pandas as pd

# Add the '15_miles_from_home' column
final_merged_df['15_miles_from_home'] = final_merged_df['distance_miles'].apply(lambda x: 'yes' if x <= 15 else 'no')

# Add the '25_miles_from_home' column
final_merged_df['25_miles_from_home'] = final_merged_df['distance_miles'].apply(lambda x: 'yes' if 15 < x <= 25 else 'no')

In [282]:
# Save the filtered DataFrame to CSV
final_merged_df.to_csv('final_list_of_var_v1.csv', index=False)

## Attach High Speed Information

In [284]:
# Merge final_merged_df and high_speed_df on 'subj' and 'drive'
final_high_speed= pd.merge(final_merged_df, high_speed_df, on=['subj', 'drive'], how='left')


In [286]:
# Rename the column
final_high_speed.rename(columns={'percentage_at_60mph_or_greater': 'high_speed_drive'}, inplace=True)

# Replace non-null values with 'Yes'
final_high_speed['high_speed_drive'] = final_high_speed['high_speed_drive'].notnull().replace({True: 'Yes', False: None})



In [288]:
final_high_speed.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,start_location,end_location,trip_chain_check,education_years,race,gender,age,AM_PM_peak,day_or_night,high_speed_drive
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,NaN,NaN,NaN,mastersdegree,white,male,79.0,None,daytime,None
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,unc,social,NaN,mastersdegree,white,male,79.0,None,daytime,None
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,unc,errand,NaN,mastersdegree,white,male,79.0,None,daytime,None
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,unc,home,NaN,mastersdegree,white,male,79.0,None,daytime,None
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,home,home,1.0,mastersdegree,white,male,79.0,None,daytime,None


## Hard Braking information

In [290]:
# Define the file path
file_path = r'C:/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Hard_Braking/final_agg_hard_braking_3551.csv'

# Read the CSV file into a DataFrame
hard_brake_df = pd.read_csv(file_path)

In [292]:
hard_brake_df.head()

,subj,drive,time_utc,agg_hard_brake
0,RWRAD_001,1,2021-03-18 17:01:39,-0.363169
1,RWRAD_001,14,2021-03-19 19:42:29,-0.358920
2,RWRAD_001,23,2021-03-23 19:57:47,-0.367135
3,RWRAD_001,26,2021-03-24 14:25:33,-0.423241
4,RWRAD_001,71,2021-04-05 18:29:58,-0.396596


In [294]:
# Group by 'subj' and 'drive', and count the occurrences of 'agg_hard_brake' for each combination
hard_brake_counts = (
    hard_brake_df.groupby(['subj', 'drive'])
    .agg(count=('agg_hard_brake', 'count'))
    .reset_index()
)

# Store the result in a DataFrame for further merging
hard_brake_counts_df = pd.DataFrame(hard_brake_counts)

In [296]:
hard_brake_counts_df.head()

,subj,drive,count
0,RWRAD_001,1,1
1,RWRAD_001,14,1
2,RWRAD_001,23,1
3,RWRAD_001,26,1
4,RWRAD_001,71,2


In [298]:
# Merge final_high_speed and hard_brake_counts_df based on 'subj' and 'drive'
merged_df = pd.merge(final_high_speed, hard_brake_counts_df, on=['subj', 'drive'], how='left')

In [300]:
merged_df.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,end_location,trip_chain_check,education_years,race,gender,age,AM_PM_peak,day_or_night,high_speed_drive,count
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,NaN,NaN,mastersdegree,white,male,79.0,None,daytime,None,1.0
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,social,NaN,mastersdegree,white,male,79.0,None,daytime,None,NaN
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,errand,NaN,mastersdegree,white,male,79.0,None,daytime,None,NaN
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,home,NaN,mastersdegree,white,male,79.0,None,daytime,None,NaN
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,home,1.0,mastersdegree,white,male,79.0,None,daytime,None,NaN


In [302]:
# Save the filtered DataFrame to CSV
merged_df.to_csv('final_list_of_var_v2.csv', index=False)

## Probability of stopping

In [459]:
# Define the file path
file_path =  r'C:/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Final_list_of_variables/final_list_of_var_v2.csv'

# Read the CSV file into a DataFrame
list_of_var = pd.read_csv(file_path)

C:\Users\aparnaj8\AppData\Local\Temp\ipykernel_10392\2613430640.py:5: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  list_of_var = pd.read_csv(file_path)


In [461]:
# Define the file path
file_path =  r'C:/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Stopping_at_stop_sign/probability_of_stopping_final.csv'

# Read the CSV file into a DataFrame
stoppage_df = pd.read_csv(file_path)

In [463]:
list_of_var.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,end_location,trip_chain_check,education_years,race,gender,age,AM_PM_peak,day_or_night,high_speed_drive,count
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,NaN,NaN,mastersdegree,white,male,79.0,NaN,daytime,NaN,1.0
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,social,NaN,mastersdegree,white,male,79.0,NaN,daytime,NaN,NaN
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,errand,NaN,mastersdegree,white,male,79.0,NaN,daytime,NaN,NaN
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,home,NaN,mastersdegree,white,male,79.0,NaN,daytime,NaN,NaN
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,home,1.0,mastersdegree,white,male,79.0,NaN,daytime,NaN,NaN


In [465]:
# Rename the 'count' column
list_of_var.rename(columns={'count': 'hard_brake_count'}, inplace=True)

In [467]:
stoppage_df.head()

,subj,week_number,total_stop_signs,stoppages_yes,stoppage_probability
0,RWRAD_002,14,1,1,1.0
1,RWRAD_002,15,1,1,1.0
2,RWRAD_002,17,1,1,1.0
3,RWRAD_002,18,2,2,1.0
4,RWRAD_002,22,2,2,1.0


## Attach probability of stopping for every subjec

In [470]:
# Merge based on 'subj' and 'week_number'
final_df = pd.merge(list_of_var, stoppage_df, on=['subj', 'week_number'], how='left')

In [472]:
final_df.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,race,gender,age,AM_PM_peak,day_or_night,high_speed_drive,hard_brake_count,total_stop_signs,stoppages_yes,stoppage_probability
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,white,male,79.0,NaN,daytime,NaN,1.0,NaN,NaN,NaN
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,white,male,79.0,NaN,daytime,NaN,NaN,NaN,NaN,NaN
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,white,male,79.0,NaN,daytime,NaN,NaN,NaN,NaN,NaN
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,white,male,79.0,NaN,daytime,NaN,NaN,NaN,NaN,NaN
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,white,male,79.0,NaN,daytime,NaN,NaN,NaN,NaN,NaN


In [474]:
# Drop the specified columns from final_df
final_df = final_df.drop(columns=['total_stop_signs', 'stoppages_yes'])


In [476]:
final_df.dtypes

subj                     object
drive                     int64
time_start_utc           object
time_end_utc             object
time_start_cst           object
time_end_cst             object
time_weekday             object
duration_minutes        float64
distance_miles          float64
speed_mph_mean          float64
15_miles_from_home       object
25_miles_from_home       object
year                      int64
year_mapped               int64
subj_type                object
week_number               int64
start_location           object
end_location             object
trip_chain_check        float64
education_years          object
race                     object
gender                   object
age                     float64
AM_PM_peak               object
day_or_night             object
high_speed_drive         object
hard_brake_count        float64
stoppage_probability    float64
dtype: object

In [478]:
len(final_df)

46267

In [480]:
import pandas as pd

# Ensure time_start_cst and time_end_cst are in datetime format
final_df['time_start_cst'] = pd.to_datetime(final_df['time_start_cst'])
final_df['time_end_cst'] = pd.to_datetime(final_df['time_end_cst'])

# Calculate the duration in minutes
final_df['calculated_minutes'] = (final_df['time_end_cst'] - final_df['time_start_cst']).dt.total_seconds() / 60

# Update distance_minutes where it is NaN and distance_miles > 0
condition = (final_df['duration_minutes'].isna()) & (final_df['distance_miles'] > 0)
final_df.loc[condition, 'duration_minutes'] = final_df.loc[condition, 'calculated_minutes']

# Drop the temporary 'calculated_minutes' column if it's no longer needed
final_df.drop(columns=['calculated_minutes'], inplace=True)


In [482]:
# Count total NaN values in the entire DataFrame
total_nan = final_df.isna().sum().sum()
print(f"Total NaN values in the DataFrame: {total_nan}")

# Count NaN values for each column
nan_per_column = final_df.isna().sum()
print("\nNaN values per column:")
print(nan_per_column)


Total NaN values in the DataFrame: 298264

NaN values per column:
subj                        0
drive                       0
time_start_utc              0
time_end_utc                0
time_start_cst              0
time_end_cst                0
time_weekday             1482
duration_minutes            1
distance_miles              0
speed_mph_mean            206
15_miles_from_home          0
25_miles_from_home          0
year                        0
year_mapped                 0
subj_type               12979
week_number                 0
start_location          14880
end_location            14588
trip_chain_check        43268
education_years         12806
race                    30534
gender                  12748
age                     12806
AM_PM_peak              30942
day_or_night                0
high_speed_drive        39937
hard_brake_count        43166
stoppage_probability    27921
dtype: int64


In [484]:
# Save the filtered DataFrame to CSV
final_df.to_csv('final_list_of_var_v3.csv', index=False)

## Final cleaning and data addition

#### remove nan from subj type

In [4]:
# Define the file path
file_path = '/Users/aparn/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Final_list_of_variables/final_list_of_var_v3.csv'

# Read the CSV file into a DataFrame
final_df = pd.read_csv(file_path)

C:\Users\aparn\AppData\Local\Temp\ipykernel_4644\3220602463.py:5: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  final_df = pd.read_csv(file_path)


In [5]:
# Count of NaN values in 'subj_type' column
nan_count = final_df['subj_type'].isna().sum()

print(f"Number of NaN values in 'subj_type': {nan_count}")


Number of NaN values in 'subj_type': 12979


In [6]:
len(final_df)

46267

In [9]:
final_df['subj'].nunique()

119

In [8]:
# Remove rows where 'subj_type' is NaN
final_df = final_df.dropna(subset=['subj_type'])


In [537]:
final_df.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,race,gender,AM_PM_peak,day_or_night,high_speed_drive,hard_brake_count,stoppage_probability,subj_type,age,year_map
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,white,male,NaN,daytime,NaN,1.0,NaN,mci,79.0,1.0
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,white,male,NaN,daytime,NaN,NaN,NaN,mci,79.0,1.0
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,white,male,NaN,daytime,NaN,NaN,NaN,mci,79.0,1.0
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,white,male,NaN,daytime,NaN,NaN,NaN,mci,79.0,1.0
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,white,male,NaN,daytime,NaN,NaN,NaN,mci,79.0,1.0


### remove 2024 data

In [539]:
# Count the rows where 'year' is 2024
count_2024 = final_df[final_df['year'] == 2024].shape[0]

# Display the count
print(f"Count of rows where year == 2024: {count_2024}")


Count of rows where year == 2024: 0


### attach acceleration infromation 

In [555]:
# Define the file path
file_path =  r'C:/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/average_accel_x_RWRAD.csv'

# Read the CSV file into a DataFrame
avg_accel = pd.read_csv(file_path)

In [557]:
avg_accel.head()

,subj,drive,average_accel_x
0,RWRAD_001,1,0.006913
1,RWRAD_001,2,-0.004334
2,RWRAD_001,3,0.005476
3,RWRAD_001,4,0.008922
4,RWRAD_001,5,0.001417


In [559]:
final_df1 = pd.merge(final_df, avg_accel, on=['subj', 'drive'], how='left')

In [561]:
final_df1.head()

,subj,drive,time_start_utc,time_end_utc,time_start_cst,time_end_cst,time_weekday,duration_minutes,distance_miles,speed_mph_mean,...,gender,AM_PM_peak,day_or_night,high_speed_drive,hard_brake_count,stoppage_probability,subj_type,age,year_map,average_accel_x
0,RWRAD_001,1,2021-03-18 16:57:30,2021-03-18 17:02:27,2021-03-18 11:57:30,2021-03-18 12:02:27,thursday,4.950000,1.801193,22.282799,...,male,NaN,daytime,NaN,1.0,NaN,mci,79.0,1.0,0.006913
1,RWRAD_001,2,2021-03-18 17:06:48,2021-03-18 17:24:06,2021-03-18 12:06:48,2021-03-18 12:24:06,thursday,17.300000,4.304489,14.928864,...,male,NaN,daytime,NaN,NaN,NaN,mci,79.0,1.0,-0.004334
2,RWRAD_001,3,2021-03-18 17:36:45,2021-03-18 17:39:55,2021-03-18 12:36:45,2021-03-18 12:39:55,thursday,3.166667,0.370846,7.026555,...,male,NaN,daytime,NaN,NaN,NaN,mci,79.0,1.0,0.005476
3,RWRAD_001,4,2021-03-18 17:59:46,2021-03-18 18:21:48,2021-03-18 12:59:46,2021-03-18 13:21:48,thursday,22.033333,6.953906,18.950842,...,male,NaN,daytime,NaN,NaN,NaN,mci,79.0,1.0,0.008922
4,RWRAD_001,5,2021-03-18 20:36:08,2021-03-18 21:23:42,2021-03-18 15:36:08,2021-03-18 16:23:42,thursday,47.566667,9.746924,13.039364,...,male,NaN,daytime,NaN,NaN,NaN,mci,79.0,1.0,0.001417


In [549]:
len(final_df1)

33288

In [589]:
final_df1.dtypes

subj                     object
drive                     int64
time_start_utc           object
time_end_utc             object
time_start_cst           object
time_end_cst             object
time_weekday             object
duration_minutes        float64
distance_miles          float64
speed_mph_mean          float64
15_miles_from_home       object
25_miles_from_home       object
year                      int64
week_number               int64
start_location           object
end_location             object
trip_chain_check        float64
education_years          object
race                     object
gender                   object
AM_PM_peak               object
day_or_night             object
high_speed_drive         object
hard_brake_count        float64
stoppage_probability    float64
subj_type                object
age                     float64
year_map                float64
average_accel_x         float64
dtype: object

## Extract final list of variables

In [593]:
weekly_stats_df = pd.DataFrame()


In [595]:
import pandas as pd
import numpy as np

# Function to calculate all the required weekly statistics
def calculate_weekly_stats(df):
    # Convert 'time_start_utc' and 'time_end_utc' to datetime format if not already
    df['time_start_utc'] = pd.to_datetime(df['time_start_utc'])
    df['time_end_utc'] = pd.to_datetime(df['time_end_utc'])
    df['time_start_cst'] = pd.to_datetime(df['time_start_cst'])
    df['time_end_cst'] = pd.to_datetime(df['time_end_cst'])
    
    # Initialize an empty dictionary to store calculated values
    weekly_stats = {}
    
    # Total number of miles driven in a week
    weekly_stats['total_miles'] = df['distance_miles'].sum()

    # Total number of trips in a week
    weekly_stats['total_trips'] = df.shape[0]

    # Total miles driven in a week divided by total number of trips in a week
    weekly_stats['avg_miles_per_trip'] = weekly_stats['total_miles'] / weekly_stats['total_trips']

    # Total driving minutes in a week
    weekly_stats['total_minutes'] = df['duration_minutes'].sum()

    # Total minutes of driving in a week divided by total number of trips in a week
    weekly_stats['avg_minutes_per_trip'] = weekly_stats['total_minutes'] / weekly_stats['total_trips']

    # Number of trips in week not classified as nighttime
    weekly_stats['non_nighttime_trips'] = df[df['day_or_night'] != 'nighttime'].shape[0]

    # Percent of trips in week not classified as nighttime
    weekly_stats['percent_non_nighttime'] = (weekly_stats['non_nighttime_trips'] / weekly_stats['total_trips']) * 100

    # Number of trips in week during 7–9 AM on weekdays
    weekly_stats['am_7_9_trips'] = df[(df['AM_PM_peak'] == 'AM Peak')].shape[0]
    
    # Percent of trips in week during 7–9 AM on weekdays
    weekly_stats['percent_am_7_9'] = (weekly_stats['am_7_9_trips'] / weekly_stats['total_trips']) * 100

    # Number of trips during which at least 80% of a trip was during nighttime in week
    weekly_stats['nighttime_80_percent'] = df[df['day_or_night'] == 'nighttime'].shape[0]

    # Percent of trips during which at least 80% of a trip was during nighttime in week
    weekly_stats['percent_nighttime_80'] = (weekly_stats['nighttime_80_percent'] / weekly_stats['total_trips']) * 100

    # Number of trips in week during 4–6 PM on weekdays
    weekly_stats['pm_4_6_trips'] = df[(df['AM_PM_peak'] == 'PM Peak')].shape[0]
    
    # Percent of trips in week during 4–6 PM on weekdays
    weekly_stats['percent_pm_4_6'] = (weekly_stats['pm_4_6_trips'] / weekly_stats['total_trips']) * 100

    # Number of trips traveled in week within 15 miles of home
    weekly_stats['within_15_miles'] = df[df['15_miles_from_home'] == 'yes'].shape[0]

    # Percent of trips traveled in week within 15 miles of home
    weekly_stats['percent_within_15_miles'] = (weekly_stats['within_15_miles'] / weekly_stats['total_trips']) * 100

    # Number of trips traveled in week within 25 miles of home
    weekly_stats['within_25_miles'] = df[df['25_miles_from_home'] == 'yes'].shape[0]

    # Percent of trips traveled in week within 25 miles of home
    weekly_stats['percent_within_25_miles'] = (weekly_stats['within_25_miles'] / weekly_stats['total_trips']) * 100

    # Number of trip chains in week (chain is a series of trips marked by unique values in 'trip_chain_check')
    weekly_stats['trip_chains'] = df['trip_chain_check'].nunique()
    
    # Calculate the total miles driven for each trip chain
    total_miles_chains = df.groupby('trip_chain_check')['distance_miles'].sum()
    
    # Calculate the total driving minutes for each trip chain
    total_minutes_chains = df.groupby('trip_chain_check')['duration_minutes'].sum()
    
    # Calculate the total number of unique trip chains
    total_trip_chains = df['trip_chain_check'].nunique()
    
    # Calculate the total miles of chains divided by the number of trip chains
    avg_miles_per_chain = total_miles_chains.sum() / total_trip_chains if total_trip_chains != 0 else np.nan
    
    # Calculate the total driving minutes for chains divided by the number of trip chains
    avg_minutes_per_chain = total_minutes_chains.sum() / total_trip_chains if total_trip_chains != 0 else np.nan
    
    # Add these to the weekly stats dictionary
    weekly_stats['avg_miles_per_chain'] = avg_miles_per_chain
    weekly_stats['avg_minutes_per_chain'] = avg_minutes_per_chain

    # Make sure we only count 'Yes' and avoid NaN values
    weekly_stats['speed_60_mph'] = df[df['high_speed_drive'] == 'Yes'].shape[0]
    
    # Percentage of trips in week where 20% of distance traveled was at a high-speed road
    # If the total trips are non-zero, calculate percentage, else return NaN
    weekly_stats['percent_speed_60_mph'] = (weekly_stats['speed_60_mph'] / weekly_stats['total_trips']) * 100 if weekly_stats['total_trips'] != 0 else np.nan

    # The average speed of trips
    weekly_stats['avg_trip_speed'] = df['speed_mph_mean'].mean()

    # The average acceleration of trips
    weekly_stats['avg_trip_accel_x'] = df['average_accel_x'].mean()
    
    # Number of events with a deceleration rate <= -0.35 g in a week
    weekly_stats['hard_brake_events'] = df[df['hard_brake_count'] > 0].shape[0]

    # Handling stoppage_probability: keeping unique values per subj
    stoppage_values = df['stoppage_probability'].dropna().unique()
    if len(stoppage_values) > 0:
        weekly_stats['stoppage_probability'] = stoppage_values[0]  # Attach first unique value for subj

    return weekly_stats

# Group by subj, week_number, and year, then apply the calculation function
weekly_stats_df = final_df1.groupby(['subj', 'week_number', 'year']).apply(calculate_weekly_stats).apply(pd.Series)

# Reset index to ensure 'subj', 'week_number', and 'year' are in the columns
weekly_stats_df.reset_index(inplace=True)

# Fill NaNs in 'stoppage_probability' column with empty strings where no value exists
weekly_stats_df['stoppage_probability'] = weekly_stats_df['stoppage_probability'].fillna('')

# Ensure 'year' information is included in the final DataFrame

# Reorder columns to have 'subj', 'week_number', and 'year' as the first three columns
cols = ['subj', 'week_number', 'year'] + [col for col in weekly_stats_df.columns if col not in ['subj', 'week_number', 'year']]
weekly_stats_df = weekly_stats_df[cols]


# Reorder columns to have 'subj' and 'week_number' as the first two columns
#cols = ['subj', 'week_number'] + [col for col in weekly_stats_df.columns if col not in ['subj', 'week_number']]
#weekly_stats_df = weekly_stats_df[cols]

C:\Users\aparnaj8\AppData\Local\Temp\ipykernel_10392\1689638292.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time_start_cst'] = pd.to_datetime(df['time_start_cst'])
C:\Users\aparnaj8\AppData\Local\Temp\ipykernel_10392\1689638292.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time_end_cst'] = pd.to_datetime(df['time_end_cst'])
C:\Users\aparnaj8\AppData\Local\Temp\ipykernel_10392\1689638292.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time_start_utc'] = pd.to_datetime(df['time_start_utc'])
C:\Users\aparnaj8\AppData\Local\Temp\ipykernel_10392\16

In [597]:
weekly_stats_df.head()

,subj,week_number,year,total_miles,total_trips,avg_miles_per_trip,total_minutes,avg_minutes_per_trip,non_nighttime_trips,percent_non_nighttime,...,percent_within_25_miles,trip_chains,avg_miles_per_chain,avg_minutes_per_chain,speed_60_mph,percent_speed_60_mph,avg_trip_speed,avg_trip_accel_x,hard_brake_events,stoppage_probability
0,RWRAD_001,11,2021,81.858703,17.0,4.815218,284.533333,16.737255,17.0,100.0,...,0.000000,2.0,11.780566,43.908333,0.0,0.000000,18.559818,0.005058,2.0,
1,RWRAD_001,12,2021,89.800539,22.0,4.081843,303.000000,13.772727,22.0,100.0,...,0.000000,2.0,7.937587,33.241667,0.0,0.000000,18.578412,0.006890,2.0,
2,RWRAD_001,13,2021,311.730769,31.0,10.055831,612.866667,19.769892,31.0,100.0,...,6.451613,6.0,7.397040,24.972222,3.0,9.677419,23.202138,0.005666,0.0,
3,RWRAD_001,14,2021,83.971941,26.0,3.229690,311.983333,11.999359,26.0,100.0,...,0.000000,2.0,4.428278,20.808333,0.0,0.000000,19.028626,0.006027,2.0,
4,RWRAD_001,15,2021,151.992413,20.0,7.599621,296.400000,14.820000,20.0,100.0,...,0.000000,1.0,2.498817,10.666667,2.0,10.000000,23.956317,0.006580,4.0,


In [599]:
# Group by 'subj' to get unique years for each subject
multi_year_subj = final_df1.groupby('subj')['year'].nunique()

# Filter subjects that have more than one unique year
multi_year_subj = multi_year_subj[multi_year_subj > 1]

# Display the subjects with more than one unique year
print(multi_year_subj)


subj
RWRAD_027    2
RWRAD_044    2
RWRAD_047    2
RWRAD_050    2
RWRAD_052    2
RWRAD_053    2
RWRAD_055    2
RWRAD_056    2
RWRAD_057    2
RWRAD_060    2
RWRAD_061    2
RWRAD_062    2
RWRAD_063    2
RWRAD_064    2
RWRAD_065    2
RWRAD_066    2
Name: year, dtype: int64


In [601]:
# Filter the data for the subjects with more than one unique year
filtered_data = final_df1[final_df1['subj'].isin(multi_year_subj.index)]

# Group by 'subj' and check if 'subj_type' is consistent across years
subj_type_changes = filtered_data.groupby('subj').agg({
    'subj_type': 'nunique'
}).reset_index()

# Filter subjects where 'subj_type' is changing (i.e., having more than 1 unique 'subj_type')
changing_subj = subj_type_changes[subj_type_changes['subj_type'] > 1]

# Display subjects where 'subj_type' is changing
print(changing_subj)


         subj  subj_type
3   RWRAD_050          2
4   RWRAD_052          2
7   RWRAD_056          2
9   RWRAD_060          2
11  RWRAD_062          2
14  RWRAD_065          2


In [603]:
# Create a subset of final_df1 with additional columns
additional_columns = ['education_years', 'race', 'gender', 'age', 'subj_type']
final_df_subset = final_df1[['subj', 'year'] + additional_columns].drop_duplicates(subset=['subj', 'year'])

# Merge the dataframes based on 'subj' and 'year'
weekly_stats_df = weekly_stats_df.merge(final_df_subset, on=['subj', 'year'], how='left')



In [605]:
weekly_stats_df.head()

,subj,week_number,year,total_miles,total_trips,avg_miles_per_trip,total_minutes,avg_minutes_per_trip,non_nighttime_trips,percent_non_nighttime,...,percent_speed_60_mph,avg_trip_speed,avg_trip_accel_x,hard_brake_events,stoppage_probability,education_years,race,gender,age,subj_type
0,RWRAD_001,11,2021,81.858703,17.0,4.815218,284.533333,16.737255,17.0,100.0,...,0.000000,18.559818,0.005058,2.0,,mastersdegree,white,male,79.0,mci
1,RWRAD_001,12,2021,89.800539,22.0,4.081843,303.000000,13.772727,22.0,100.0,...,0.000000,18.578412,0.006890,2.0,,mastersdegree,white,male,79.0,mci
2,RWRAD_001,13,2021,311.730769,31.0,10.055831,612.866667,19.769892,31.0,100.0,...,9.677419,23.202138,0.005666,0.0,,mastersdegree,white,male,79.0,mci
3,RWRAD_001,14,2021,83.971941,26.0,3.229690,311.983333,11.999359,26.0,100.0,...,0.000000,19.028626,0.006027,2.0,,mastersdegree,white,male,79.0,mci
4,RWRAD_001,15,2021,151.992413,20.0,7.599621,296.400000,14.820000,20.0,100.0,...,10.000000,23.956317,0.006580,4.0,,mastersdegree,white,male,79.0,mci


In [607]:
len(weekly_stats_df)

1532

In [609]:
# Save the final weekly statistics dataframe to a CSV file
weekly_stats_df.to_csv('weekly_stats_with_demo.csv', index=False)